<a href="https://colab.research.google.com/github/KanmaniKannathasan/DataMining/blob/main/KMap_Classwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 872.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.5 MB/s eta 0:00:00
  Created wheel for umap-learn: filename=umap_learn-0.5.5-py3-none-any.whl size=86832 sha256=e49bb8e393d8415a90c2505f1115219cb0c7e2c4037d8a98a908054253486db0
  Stored in directory: /root/.cache/pip/wheels/3a/70/07/428d2b58660a1a3b431db59b806a10da736612ebbc66c1bcc5
Successfully built umap-learn


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import plotly.graph_objs as go
import plotly.figure_factory as ff
import umap
import re, nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
import joblib
from sklearn.cluster import KMeans

# Importing dataset and examining it
dataset = pd.read_csv("/content/drive/MyDrive/DM/Sales_Transactions_Dataset_Weekly.csv")
pd.set_option('display.max_columns', None) # to make sure you can see all the columns in output window
# print(dataset.head())
# print(dataset.shape)
print(dataset.info(verbose=True, null_counts=True))
#print(dataset.describe())


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 811 entries, 0 to 810
Data columns (total 107 columns):
 #    Column         Non-Null Count  Dtype  
---   ------         --------------  -----  
 0    Product_Code   811 non-null    object 
 1    W0             811 non-null    int64  
 2    W1             811 non-null    int64  
 3    W2             811 non-null    int64  
 4    W3             811 non-null    int64  
 5    W4             811 non-null    int64  
 6    W5             811 non-null    int64  
 7    W6             811 non-null    int64  
 8    W7             811 non-null    int64  
 9    W8             811 non-null    int64  
 10   W9             811 non-null    int64  
 11   W10            811 non-null    int64  
 12   W11            811 non-null    int64  
 13   W12            811 non-null    int64  
 14   W13            811 non-null    int64  
 15   W14            811 non-null    int64  
 16   W15            811 non-null    int64  
 17   W16            811 non-null    in

<ipython-input-3-6844c8f2c8b5>:30: FutureWarning: null_counts is deprecated. Use show_counts instead
  print(dataset.info(verbose=True, null_counts=True))


In [ ]:
# Defining feature set
X = dataset.drop(['Product_Code'], axis = 1)  # Features
print(type(X))
print(X.shape)


<class 'pandas.core.frame.DataFrame'>
(811, 54)


In [ ]:
# Normalizing numerical features so that each feature has mean 0 and variance 1
feature_scaler = StandardScaler()
X_scaled = feature_scaler.fit_transform(X)

In [ ]:
# Implementing PCA to visualize dataset
pca = PCA(n_components = 2)
pca.fit(X_scaled)
x_pca = pca.transform(X_scaled)
print("Variance explained by each of the n_components: ",pca.explained_variance_ratio_)
print("Total variance explained by the n_components: ",sum(pca.explained_variance_ratio_))
Product_Code=list(dataset['Product_Code'])
data = [go.Scatter(x=x_pca[:,0], y=x_pca[:,1], mode='markers',
                    marker = dict(color=X, colorscale='Rainbow', opacity=0.5),
                                text=[f'Product_Code: {a}' for a in Product_Code],
                                hoverinfo='text')]

layout = go.Layout(title = 'PCA Dimensionality Reduction', width = 700, height = 700,
                    xaxis = dict(title='First Principal Component'),
                    yaxis = dict(title='Second Principal Component'))
fig = go.Figure(data=data, layout=layout)
fig.show()

Product_Code=list(dataset['Product_Code'])

Variance explained by each of the n_components:  [0.92068087 0.01048955]
Total variance explained by the n_components:  0.9311704254140845


In [ ]:
# Implementing UMAP to visualize dataset
u = umap.UMAP(n_components = 2, n_neighbors=20, min_dist=0.1)
x_umap = u.fit_transform(X_scaled)
Product_Code=list(dataset['Product_Code'])
data = [go.Scatter(x=x_umap[:,0], y=x_umap[:,1], mode='markers',
                    marker = dict(color=X, colorscale='Rainbow', opacity=0.5),
                                text=[f'Product_Code: {a}' for a in Product_Code],
                                hoverinfo='text')]

layout = go.Layout(title = 'UMAP Dimensionality Reduction', width = 700, height = 700,
                    xaxis = dict(title='First Dimension'),
                    yaxis = dict(title='Second Dimension'))
fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
kmeans = KMeans(n_clusters = 3)
kmeans.fit(x_pca)
Product_Code=list(kmeans.labels_)
data = [go.Scatter(x=x_pca[:,0], y=x_pca[:,1], mode='markers',
                    marker = dict(color=kmeans.labels_, colorscale='Rainbow', opacity=0.5),
                                text=[f'Product_Code: {a}' for a in Product_Code],
                                hoverinfo='text')]

layout = go.Layout(title = 'Data with KMeans Labels', width = 500, height = 500,
                    xaxis = dict(title='First Principal Component'),
                    yaxis = dict(title='Second Principal Component'))
fig = go.Figure(data=data, layout=layout)
fig.show()